# 🤖 ReAct Agent 만들어보기

## 📚 학습 목표
오늘은 **생각하고 행동하는** 똑똑한 AI 에이전트를 만들어보겠습니다!

### 🎯 배울 내용
- **ReAct 패턴**: Think → Act → Observe (생각→행동→관찰)
- **Microsoft Phi-4**: 가볍고 똑똑한 AI 모델
- **도구 사용**: 웹 검색과 문서 검색
- **실전 적용**: 질문하면 스스로 정보를 찾아서 답변

### ⏰ 소요시간: 25분

### 🌟 특징
- **완전 무료**: Microsoft Phi-4 오픈소스 모델
- **즉시 실행**: 서버 설치 없이 Jupyter에서 바로 사용
- **한국어 지원**: 자연스러운 한국어 대화

# 1. 📦 라이브러리 설치

**Microsoft Phi-4**와 **ReAct Agent**에 필요한 라이브러리를 설치합니다.

### 설치할 도구들
- **Microsoft Phi-4**: 가벼운 최신 AI 모델 (2.7B)
- **LangGraph**: AI 워크플로우 도구
- **검색 도구**: 웹 검색과 문서 검색

In [ ]:
# 라이브러리 설치

print("📦 필수 라이브러리들을 설치하고 있습니다...")

import subprocess
import sys

packages = [
    "torch", "transformers", "accelerate",
    "langgraph", "langchain", "langchain-community", "langchain-huggingface",
    "sentence-transformers", "faiss-cpu", "tavily-python", "langchain-teddynote"
]

for package in packages:
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                      check=True, capture_output=True)
        print(f"✅ {package}")
    except:
        print(f"⚠️ {package} 설치 실패")

print("\n✅ 설치 완료! 이제 AI 모델을 설정하겠습니다.")

In [ ]:
# 필수 도구들 불러오기

import warnings
warnings.filterwarnings('ignore')

# AI 모델과 도구들
import torch
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langgraph.prebuilt import create_react_agent

# 스트리밍 출력용 (핵심!)
from langchain_teddynote.messages import stream_graph

# 검색 도구들
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_core.tools.retriever import create_retriever_tool

print("✅ 모든 도구를 성공적으로 불러왔습니다!")
print("🌟 stream_graph 방식으로 깔끔한 출력 지원!")

# 2. 🧠 Microsoft Phi-4 AI 모델 설정

**Microsoft Phi-4**는 작지만 똑똑한 최신 AI 모델입니다.
- **크기**: 2.7B 파라미터 (가벼움)
- **특징**: 지시문 이해와 추론에 뛰어남
- **장점**: 한국어 지원, ReAct 패턴에 최적화

### 🔧 ChatHuggingFace 래퍼 사용
`HuggingFacePipeline`은 `bind_tools`를 지원하지 않아서 `ChatHuggingFace`로 래핑해야 합니다.

In [ ]:
# Microsoft Phi-4 모델 설정

print("🧠 Microsoft Phi-4 모델을 로드하고 있습니다...")

# 1단계: HuggingFace Pipeline 생성
hf_pipeline = pipeline(
    "text-generation",
    model="microsoft/Phi-4-mini-instruct",
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    max_new_tokens=200,
    temperature=0.7,
    pad_token_id=50256
)

# 2단계: HuggingFacePipeline 래퍼
base_llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 3단계: ChatHuggingFace로 래핑 (bind_tools 지원)
llm = ChatHuggingFace(llm=base_llm)

print("✅ Microsoft Phi-4 모델 준비 완료!")
print("💡 ChatHuggingFace 래퍼로 bind_tools 지원 활성화!")

# 3. 🛠️ AI가 사용할 도구들 만들기

AI가 다양한 정보를 찾을 수 있도록 **2가지 도구**를 준비합니다.

### 도구 1: 웹 검색
- 최신 정보를 인터넷에서 검색
- 실시간 뉴스, 날씨, 트렌드 등

### 도구 2: 문서 검색  
- 내부 문서에서 정보 검색
- AI, LangGraph 관련 지식

In [ ]:
# 웹 검색 도구 설정

import os

# Tavily API 키 설정 (무료 웹 검색 서비스)
os.environ["TAVILY_API_KEY"] = "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"

# 웹 검색 도구 생성
web_search = TavilySearch(
    max_results=3,
    search_depth="basic",
    include_answer=True
)
web_search.name = "web_search"
web_search.description = """최신 정보, 실시간 데이터, 뉴스, 날씨, 트렌드를 인터넷에서 검색하는 도구입니다.
사용 시기: 최신, 동향, 뉴스, 날씨, 2024년, 현재, 실시간 정보가 필요할 때"""

print("✅ 웹 검색 도구 준비 완료!")

In [ ]:
# 문서 검색 도구 설정

# 문서 임베딩 모델 
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# 예시 문서들 (AI 관련 지식)
sample_docs = [
    "LangGraph는 AI 워크플로우를 그래프 형태로 만드는 도구입니다. 복잡한 AI 작업을 단계별로 나눠서 처리할 수 있어요.",
    "ReAct는 Reasoning(생각)과 Acting(행동)을 결합한 AI 패턴입니다. Think → Act → Observe 순서로 동작해요.",
    "StateGraph는 LangGraph의 핵심으로, 각 단계의 상태를 관리하며 워크플로우를 실행합니다.",
    "Microsoft Phi-4는 작지만 뛰어난 성능을 가진 언어 모델입니다. 지시문 이해에 특화되어 있어요."
]

# 벡터 저장소 생성
docs = [Document(page_content=text) for text in sample_docs]
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 문서 검색 도구 생성
doc_search = create_retriever_tool(
    retriever,
    name="document_search",
    description="""AI 기술, LangGraph, ReAct, StateGraph 등 AI 관련 내부 문서에서 정보를 검색하는 도구입니다.
사용 시기: ReAct, LangGraph, StateGraph, AI 패턴, 메모리, Microsoft Phi-4 등 AI 기술에 대한 질문일 때"""
)

print("✅ 문서 검색 도구 준비 완료!")

# 4. 🔄 ReAct 패턴이란?

## ReAct = **Re**asoning + **Act**ing

### 3단계 순환 과정
1. **🤔 Think (생각하기)**: 질문을 분석하고 전략 수립
2. **🛠️ Act (행동하기)**: 필요한 도구를 선택해서 정보 수집
3. **👀 Observe (관찰하기)**: 결과를 분석하고 다음 행동 결정

### 예시: "오늘 날씨는?"
1. **Think**: 현재 날씨 정보가 필요하다 → 웹 검색 사용
2. **Act**: 웹 검색 도구로 "오늘 날씨" 검색  
3. **Observe**: 검색 결과를 보고 "서울 20도, 맑음"이라고 답변

# 5. 🚀 ReAct Agent 만들기

## 핵심: create_react_agent + MemorySaver

**LangGraph**의 `create_react_agent`를 사용하면 복잡한 구현 없이 간단하게 ReAct 패턴을 구현할 수 있습니다.

### 🧠 MemorySaver: 대화 기억의 핵심

**MemorySaver**는 LangGraph의 메모리 저장소입니다:
- **checkpointer**: AI가 이전 대화를 기억할 수 있게 해주는 핵심 기능
- **thread_id**: 각 대화 세션을 구분하는 고유 ID
- **자동 저장**: 각 대화 턴마다 자동으로 상태를 저장

### 🔄 메모리 없는 AI vs 메모리 있는 AI

| 구분 | 메모리 없음 | MemorySaver 있음 |
|------|-------------|-------------------|
| 대화 기억 | ❌ 매번 처음부터 | ✅ 이전 대화 완벽 기억 |
| 사용자 정보 | ❌ 기억 못함 | ✅ 이름, 선호도 기억 |
| 대화 흐름 | ❌ 단절적 | ✅ 자연스러운 연속성 |
| 교육 효과 | ❌ 제한적 | ✅ 실제 AI 체험 |

In [ ]:
# ReAct Agent 생성 (MemorySaver로 메모리 관리!)

# 🧠 핵심: MemorySaver 메모리 저장소 설정
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소 생성 (대화 기억 기능)
memory = MemorySaver()
print("🧠 MemorySaver 메모리 저장소 생성 완료!")

# 도구 목록 준비
tools = [web_search, doc_search]

# 🌟 메모리 기반 ReAct Agent 생성 (checkpointer 추가!)
react_agent = create_react_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,  # 🚀 핵심: 메모리 저장소 연결!
    prompt="""당신은 친근하고 도움이 되는 AI 어시스턴트입니다. 사용자의 언어에 맞춰 응답하세요.

🚫 도구를 사용하지 않는 경우:
- 인사말, 자기소개, 일반적인 대화
- 이미 알고 있는 기본적인 정보나 상식
- 단순한 질문이나 개인적인 대화

🔍 도구 사용 가이드:
- 특정 AI 기술 질문 (ReAct, LangGraph, StateGraph, Microsoft Phi-4) → document_search
- 최신 정보 필요 (뉴스, 날씨, 트렌드, 2024년 이후 정보) → web_search

💬 대화 원칙:
- 자연스럽고 친근한 대화를 우선시
- 이전 대화 내용을 기억하여 맥락 유지
- 필요한 경우에만 도구 사용
- 사용자의 언어(한국어/영어)로 응답"""
)

# 🌟 MemorySaver 기반 멀티턴 대화 함수
def simple_react(question):
    """MemorySaver로 완벽한 멀티턴 대화 지원!"""
    
    # ✨ 메모리 기반 Config 설정
    config = {
        "configurable": {
            "thread_id": "conversation"  # 대화 스레드 ID
        }
    }
    
    # 📝 입력 설정
    inputs = {
        "messages": [("human", question)]
    }
    
    print(f"👤 질문: {question}")
    print("\n🤖 답변:")
    
    # 🚀 핵심: MemorySaver + stream_graph = 완벽한 메모리!
    try:
        stream_graph(react_agent, inputs, config)
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        print("🔄 다시 시도해주세요.")

print("✅ ReAct Agent 준비 완료!")
print("🧠 MemorySaver로 완벽한 멀티턴 대화 지원!")
print("🌟 이제 AI가 이전 대화를 완벽하게 기억합니다!")

# 6. 💬 ReAct Agent 테스트

## 테스트 1: 문서 검색

In [ ]:
# 문서 검색 테스트

simple_react("ReAct 패턴이 무엇인지 설명해주세요")

# 테스트 2: 웹 검색

In [ ]:
# 웹 검색 테스트

simple_react("2024년 AI 기술 동향을 알려주세요")

# 테스트 3: 일반 대화

In [ ]:
# 멀티턴 대화 테스트 (MemorySaver 기반)

print("🧪 테스트 3-1: 자기소개 (메모리 저장)")
print("💡 예상: 도구 사용 없이 친근한 인사, 이름을 기억에 저장")
simple_react("안녕하세요! 나의 이름은 라이언이야.")

print("\n" + "="*60)
print("🧪 테스트 3-2: 멀티턴 대화 (메모리 테스트)")
print("💡 예상: MemorySaver가 '테디'라는 이름을 완벽하게 기억하고 답변")
simple_react("내 이름이 뭐라고 했죠?")


# 7. 🎮 직접 해보세요!

## 자유 테스트

아래에서 `simple_react("질문")` 형태로 다양한 질문을 해보세요!

### 추천 질문들
```python
# 문서 검색 질문
simple_react("LangGraph는 무엇인가요?")

# 웹 검색 질문  
simple_react("오늘 날씨는 어때요?")

# 일반 대화
simple_react("프로그래밍 공부 팁을 알려주세요")
```

In [ ]:
# 자유 테스트 공간 (stream_graph 방식)

print("🎮 자유롭게 테스트해보세요!")
print("💡 예시:")
print('simple_react("질문을 여기에 입력하세요")')
print()

# 여기서 테스트해보세요! 👇

# 추천 테스트 질문들:
# simple_react("LangGraph가 무엇인가요?")  # 문서 검색
# simple_react("오늘 날씨는 어때요?")        # 웹 검색  
# simple_react("안녕하세요!")               # 일반 대화

print("🌟 stream_graph 방식으로 실시간 스트리밍을 확인하세요!")

# 🎓 정리 - MemorySaver의 혁신!

## 🌟 완성한 것

### ✅ 혁신적 개선사항
- **🧠 MemorySaver**: 완벽한 멀티턴 대화 기억 시스템
- **🌊 stream_graph**: 실시간 스트리밍 출력으로 깔끔한 결과
- **🔧 checkpointer**: LangGraph의 공식 메모리 관리 시스템
- **🛠️ 도구 통합**: 웹 검색 + 문서 검색 완벽 연동
- **💬 자연스러운 대화**: 진짜 AI 어시스턴트 같은 경험

### 🔥 핵심 혁신 코드
```python
# 🧠 MemorySaver: 대화 기억의 핵심
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# 🌟 메모리 기반 ReAct Agent
react_agent = create_react_agent(
    model=llm,
    tools=tools,
    checkpointer=memory,  # 🚀 완벽한 메모리!
    prompt="""..."""
)

# 🔄 멀티턴 대화 함수
def simple_react(question):
    config = {"configurable": {"thread_id": "conversation"}}
    inputs = {"messages": [("human", question)]}
    stream_graph(react_agent, inputs, config)
```

